# **Detecting Logo of "جمهورية مصر العربية : بطاقة تحقيق الشخصية" in Egyptian IDs and Inpainting It**

## **1) First Method:** Keras OCR



* Here, I'm just using keras ocr ready tool for object detection without any fine
tuning
* After detecting the bounding box , we inpaint it using background color

In [ ]:
#Getting the data in runtime
!git clone https://github.com/Nagwa-Ahmed/Logo-Detection-in-Egyptian-IDs.git

Cloning into 'Logo-Detection-in-Egyptian-IDs'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (21/21), 3.83 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [1]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.

In [1]:
#Installing all libraries needed in the notebook
!pip install keras-ocr
!pip install numpy
!pip install opencv-python
#!pip install os
#!pip install glob
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 7.5 MB/s eta 0:00:00


In [2]:
#Importing required libraries
import os
import glob
import cv2
import keras_ocr
import numpy as np
from ultralytics import YOLO

In [ ]:
# Define the path to the directory containing images
image_dir = '/content/Logo-Detection-in-Egyptian-IDs/input'

# Use glob to list all image files in the directory
image_paths = glob.glob(os.path.join(image_dir, '*.*'))

#output images will be saved in the folder "Output"
!rm -rf Output
!mkdir Output


In [ ]:
for image_path in image_paths:

  # Initializing the Keras OCR pipeline
  pipeline = keras_ocr.pipeline.Pipeline()

  image = keras_ocr.tools.read(image_path)

  # Detecting text in the image
  prediction_groups = pipeline.recognize([image])

  text_block_to_remove = prediction_groups[0][0]
  coordinates = text_block_to_remove[1]
  # Increasing the size of the rectangle by a certain number of pixels since at the first time the rectangle was so small
  padding = 20

  # Calculate the minimum x-coordinate from the list of coordinates and subtract padding
  x_min = int(min(coordinates, key=lambda x: x[0])[0]) - padding
  y_min = int(min(coordinates, key=lambda x: x[1])[1]) - padding

  # Calculate the maximum x-coordinate from the list of coordinates and add padding
  x_max = int(max(coordinates, key=lambda x: x[0])[0]) + padding
  y_max = int(max(coordinates, key=lambda x: x[1])[1]) + padding

  # Create a larger mask for the selected text block
  mask = np.zeros(image.shape[:2], dtype=np.uint8)
  cv2.rectangle(mask, (x_min, y_min), (x_max, y_max), (255, 255, 255), -1)


  # Inpainting the masked area
  inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)

  inpainted_image_rgb = cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB)

  # Saveing the inpainted image
  cv2.imwrite(f'/content/Output/{image_path[-6:-4]}inpainted.jpg', inpainted_image_rgb)

  #output image name isn't the best but we can proceed with it :) :)


Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 5s 5s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 6s 6s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


1/1 [==============================] - 14s 14s/step


1/1 [==============================] - 5s 5s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 4s 4s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 3s 3s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 6s 6s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 6s 6s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 6s 6s/step
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 6s 6s/step


It seems that even increasing the rectangle size couldn't improve the inpainting as expected so there are many alternatives:
* To address this problem for IDs that has a background that causes problems in inpainting such as "15" image we can run a model that detects the ID itself and crop it from the background and then run our keras ocr and it may provide better results

Unfortunately ,I couldn't apply this alternative but it seems promising for me :)
* Finetuning a model such as YOLO

It goes without saying that the second alternative needs more data (i.e. IDs ) so we have used a dataset from Kaggle
[Dataset Link](https://www.kaggle.com/datasets/mostafaebrahiem/egyptian-ids)

It has almost the same examples provided by as well as other images so it's used for finetuining

## **1) Second Method:** Finetuning YOLOv8


All the IDs in this dataset were split into 3 parts (train , validation and test) and they are annotated manually using roboflow
test images are the images provided in the examples file, sure to prevent overfitting


In [3]:
# we run this code snippet to get the data from roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="hWm0puaFCdlch5xlNIP0")
project = rf.workspace("dsiti").project("efg-abkzc")
version = project.version(3)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.18, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to efg-3 in yolov8:: 100%|██████████| 258/258 [00:00<00:00, 4064.35it/s]


In [4]:
# creating the model object
model=YOLO('yolov8m.pt')

100%|██████████| 49.7M/49.7M [00:00<00:00, 108MB/s]


<font color='red'> To run the following cell, you need to update paths of train and valid in **data.yaml**  </font>
to be as follows
`train: /content/efg-3/train/images`
`val: /content/efg-3/valid/images`
data.yaml exists in efg3 folder



In [6]:
#Training(i.e. finetuning ) YOLO
model.train(data='/content/efg-3/data.yaml',epochs=5)  # sure you can increase number of epochs but it can take more time and this may exceed colab free limi

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/efg-3/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 755k/755k [00:00<00:00, 24.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192, 192, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 6.23M/6.23M [00:00<00:00, 92.3MB/s]


...

        5/5      7.33G     0.9071     0.7525      1.237         10        640: 100%|██████████| 7/7 [00:03<00:00,  2.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.28it/s]


                   all         12         12      0.594          1      0.844      0.619

5 epochs completed in 0.011 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.36it/s]


                   all         12         12      0.988          1      0.995      0.776
Speed: 0.2ms preprocess, 9.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c477909d9f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

After training , You could find the training results in the folder run/detect/train


In [8]:
#predicting the test set
infer=YOLO('/content/runs/detect/train/weights/best.pt')
infer.predict('/content/efg-3/test/images',save=True)


image 1/9 /content/efg-3/test/images/15_jpg.rf.1036f777ac8e7db60101c222f946ad89.jpg: 640x640 1 ids, 38.1ms
image 2/9 /content/efg-3/test/images/21_jpg.rf.b06e0df3bf4da84b1b55fa21dc8e1dde.jpg: 640x640 1 ids, 35.9ms
image 3/9 /content/efg-3/test/images/2_jpg.rf.75c33262be1cf23cfa385136b9aa001f.jpg: 640x640 2 idss, 30.6ms
image 4/9 /content/efg-3/test/images/44_jpg.rf.c6b0b689232b1c61ad1aa307da216230.jpg: 640x640 1 ids, 30.7ms
image 5/9 /content/efg-3/test/images/48_jpg.rf.8a8f2ff75e5b02d46ef4a1e474121a78.jpg: 640x640 1 ids, 30.6ms
image 6/9 /content/efg-3/test/images/51_jpg.rf.634a3d69f3441485f82c69d103ceaa5a.jpg: 640x640 1 ids, 25.0ms
image 7/9 /content/efg-3/test/images/52_jpg.rf.a19712a12453ca70fcef3c34235d574f.jpg: 640x640 2 idss, 25.0ms
image 8/9 /content/efg-3/test/images/55_jpg.rf.a1dfac735d780d879eaf18ae2ef72e24.jpg: 640x640 1 ids, 25.0ms
image 9/9 /content/efg-3/test/images/9_jpg.rf.dad7b871ba73024e336de85e597ac11f.jpg: 640x640 1 ids, 24.4ms
Speed: 2.9ms preprocess, 29.5ms infe

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ids'}
 obb: None
 orig_img: array([[[45, 41, 36],
         [43, 39, 34],
         [43, 39, 34],
         ...,
         [ 9,  6,  2],
         [ 9,  6,  2],
         [ 8,  5,  1]],
 
        [[44, 40, 35],
         [42, 38, 33],
         [41, 37, 32],
         ...,
         [10,  7,  3],
         [10,  7,  3],
         [ 9,  6,  2]],
 
        [[46, 42, 37],
         [44, 40, 35],
         [42, 38, 33],
         ...,
         [11,  8,  4],
         [11,  8,  4],
         [11,  8,  4]],
 
        ...,
 
        [[13, 10,  6],
         [14, 11,  7],
         [13, 10,  6],
         ...,
         [ 9,  5,  4],
         [ 9,  5,  4],
         [ 9,  5,  4]],
 
        [[10,  7,  3],
         [13, 10,  6],
         [13, 10,  6],
         ...,
         [ 9,  5,  4],
         [ 9,  5,  4],
         [ 9,  5,  4]],
 
        [[13, 10,  6],
       

Prediction resluts are saved in run/detect/train/predict
you can view the bounding boxes for images are good so far although some images have two bounding boxes which is a problem
==>
* If they aren't overlapping and we are sure one of them is the correct one we can choose it as it's usually has the highest confidence
* If they are overlapping and none of them is correct then it would be better if we retrain the model with more data or more epochs may be this may improve its performance


<font color='red'>After predicting the bounding box then we need to inpaint this bounding box using background color
but unforutnately all my trials to do that couldn't acheive this </font>

Model weihgts can't be uploaded to github due to size limit so it will be uploaded to drive
You can use them to predict directly without retraining the model , just add them here to the runtime and adjust the path
[Weights link](https://drive.google.com/drive/folders/11aLE6QDlpltnHPnFbyfZdHLJlqV2Kuz8?usp=sharing)